<a href="https://colab.research.google.com/github/SiaZin/2024-Ukraine-massive-shelling-analysis/blob/main/asyncio_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Отримання даних про погоду під час обстрілів
<p>На вхід подаються дати Test_dates = ["2024-06-14","2024-07-08", "2024-10-22"]
<p>На виході отримуєм pandas dataframe
<p>Наразі список дат неповний

In [1]:
!git clone https://github.com/SiaZin/2024-Ukraine-massive-shelling-analysis.git

Cloning into '2024-Ukraine-massive-shelling-analysis'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 4.34 KiB | 4.34 MiB/s, done.


In [ ]:
import asyncio
import time
import aiohttp
from aiohttp import ClientSession

import nest_asyncio
nest_asyncio.apply()

import pandas as pd

In [ ]:
import datetime
import pytz  # Для точного врахування часових поясів

#Так яу обстріли проходять зранку, вимір темпаратури буде відносно 8 ранку

def get_utc_timestamp(date_str, time_str='08:00'):
    """
    Перетворює дату і час в форматі 'YYYY-MM-DD' і 'HH:MM' на Unix timestamp, UTC time zone (dt API call parameter).
    """
    local = pytz.timezone('Europe/Kiev')
    local_time = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
    local_time = local.localize(local_time)  # Прив'язка до локального часового поясу
    utc_time = local_time.astimezone(pytz.utc)  # Конвертація в UTC
    return int(utc_time.timestamp())

# Тест функції
print(get_utc_timestamp("2024-06-14", "08:00"))

1718341200


In [ ]:
'''
DATES = [
    {'date': '2024-06-14 08:00', 'timestamp': int(time.mktime(time.strptime('2024-06-14 08:00', '%Y-%m-%d %H:%M')))},
    {'date': '2024-07-08 08:00', 'timestamp': int(time.mktime(time.strptime('2024-07-08 08:00', '%Y-%m-%d %H:%M')))},
    {'date': '2024-10-22 08:00', 'timestamp': int(time.mktime(time.strptime('2024-10-22 08:00', '%Y-%m-%d %H:%M')))},
]
'''

Test_dates = ["2024-06-14","2024-07-08", "2024-10-22"]

# Дати та час для запитів
DATES = []

for date in Test_dates:
    utc_timestamp = get_utc_timestamp(date)
    DATES.append({'date': f'{date} 08:00', 'timestamp': utc_timestamp})

# Перевірка результату
print(DATES)

[{'date': '2024-06-14 08:00', 'timestamp': 1718341200}, {'date': '2024-07-08 08:00', 'timestamp': 1720414800}, {'date': '2024-10-22 08:00', 'timestamp': 1729573200}]


In [ ]:
# Мій API ключ для OpenWeatherMap з доступом до One Call API 3.0.Stored in Google Colab secrets
API_KEY = '***'

# Список обласних центрів України, які знаходяться під контролем уряду
'''
CITIES = [
    {'city': 'Kyiv', 'lat': 50.4501, 'lon': 30.5234},
    {'city': 'Lviv', 'lat': 49.8397, 'lon': 24.0297},
    {'city': 'Odesa', 'lat': 46.4825, 'lon': 30.7233},
    # для тесту
]
'''

CITIES = [
    {'city': 'Kyiv', 'lat': 50.4501, 'lon': 30.5234},
    {'city': 'Lviv', 'lat': 49.8397, 'lon': 24.0297},
    {'city': 'Odesa', 'lat': 46.4825, 'lon': 30.7233},
    {'city': 'Kharkiv', 'lat': 49.9935, 'lon': 36.2304},
    {'city': 'Dnipro', 'lat': 48.4647, 'lon': 35.0462},
    {'city': 'Zaporizhzhia', 'lat': 47.8388, 'lon': 35.1396},
    {'city': 'Mykolaiv', 'lat': 46.9750, 'lon': 31.9946},
    {'city': 'Kherson', 'lat': 46.6354, 'lon': 32.6169},
    {'city': 'Poltava', 'lat': 49.5883, 'lon': 34.5514},
    {'city': 'Vinnytsia', 'lat': 49.2328, 'lon': 28.4809},
    {'city': 'Chernihiv', 'lat': 51.4982, 'lon': 31.2893},
    {'city': 'Sumy', 'lat': 50.9077, 'lon': 34.7981},
    {'city': 'Cherkasy', 'lat': 49.4285, 'lon': 32.0621},
    {'city': 'Zhytomyr', 'lat': 50.2547, 'lon': 28.6587},
    {'city': 'Chernivtsi', 'lat': 48.2915, 'lon': 25.9403},
    {'city': 'Ivano-Frankivsk', 'lat': 48.9226, 'lon': 24.7104},
    {'city': 'Ternopil', 'lat': 49.5535, 'lon': 25.5948},
    {'city': 'Lutsk', 'lat': 50.7472, 'lon': 25.3254},
    {'city': 'Rivne', 'lat': 50.6199, 'lon': 26.2516},
    {'city': 'Uzhhorod', 'lat': 48.6210, 'lon': 22.2879},
]

# Функція для отримання даних про погоду для міста та дати
async def get_weather_for_city_and_date(session, city, lat, lon, date, timestamp):
    url = 'https://api.openweathermap.org/data/3.0/onecall/timemachine'
    params = {
        'lat': lat,
        'lon': lon,
        'dt': timestamp,
        'appid': API_KEY,
        'units': 'metric'  # Одиниці виміру: Цельсії
    }

    async with session.get(url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            # Відбираємо перший запис погоди з поля "data"
            weather = data['data'][0]
            return {
                'city': city,
                'date': date,
                'temperature': weather['temp'],
                'description': weather['weather'][0]['description'],
            }
        else:
            return {
                'city': city,
                'date': date,
                'temperature': None,
                'description': None
            }

# Функція для збирання даних про погоду
async def collect_weather_data(cities, dates):
    async with ClientSession() as session:
        tasks = []
        for date_info in dates:
            for city_info in cities:
                tasks.append(get_weather_for_city_and_date(
                    session,
                    city_info['city'],
                    city_info['lat'],
                    city_info['lon'],
                    date_info['date'],
                    date_info['timestamp']
                ))

        results = await asyncio.gather(*tasks)
        return results

# Основна функція
async def main():
    weather_data = await collect_weather_data(CITIES, DATES)
    df = pd.DataFrame(weather_data)
    #print(df)
    return df

# Запуск
weather_df = asyncio.run(main())


#Перевірка
print(weather_df)


    city              date  temperature       description
0   Kyiv  2024-06-14 08:00        18.87   overcast clouds
1   Lviv  2024-06-14 08:00        11.91   overcast clouds
2  Odesa  2024-06-14 08:00        20.65   overcast clouds
3   Kyiv  2024-07-08 08:00        23.42  scattered clouds
4   Lviv  2024-07-08 08:00        21.33  scattered clouds
5  Odesa  2024-07-08 08:00        26.04         clear sky
6   Kyiv  2024-10-22 08:00         3.28     broken clouds
7   Lviv  2024-10-22 08:00         0.48   overcast clouds
8  Odesa  2024-10-22 08:00         6.71         clear sky
